# Data Processing

In [27]:
!pip install swifter

In [28]:
import pandas as pd
import swifter
import numpy as np

## CulinaryDB

### Read Culinary DB Data
The database consists of 4 csv (comma separated values) files, provided as a bundle in a .zip file.

> https://cosylab.iiitd.edu.in/culinarydb/

* **01_Recipe_Details.csv**: Details of all recipes in the database. Structure: A unique 'Recipe ID', 'Recipe Title', 'Source' (from which it was fetched), and the 'Regional Cuisine' to which it belongs.

* **02_Ingredients.csv**: List of all ingredients present in the database. Structure: 'Name of Aliased Ingredient', its 'Synonyms', a unique 'Entity ID' and the 'Category' to which the ingredient belongs.
   Note 1: Synonyms were used to alias ingredients from recipes.
   Note 2: Certain ingredients' names have been suffixed & preffixed with "#" qualifier. This has done to prevent them from participating in the aliasing process as they've been included in/as one of the 'Compound Ingredients'.
   Note 3: Certain ingredients' names have "=" wildcard. It signifies a possible space-character in its position.

* **03_Compound_Ingredients.csv**: Description of "Compound Ingredients". Structure: name of the 'Compound Ingredient', its 'Synonyms', a unique 'Entity ID', 'Components' that constitute a compound ingredient and the 'Category' of each compound ingredient.

* **04_Recipe-Ingredients_Aliases.csv**: Enumerates the mapping of each ingredient of each recipe to one of the entities in our database. Structure: A 'Recipe ID', 'Original Ingredient Name' (as in the recipe), 'Aliased Ingredient Name (from FlavorDB or Compound Ingredient)' and its 'Entity ID'.

In [29]:
recipe_details_path = "CulinaryDB/01_Recipe_Details.csv"
ingredients_path = "CulinaryDB/02_Ingredients.csv"
compound_ingredients_path = "CulinaryDB/03_Compound_Ingredients.csv"
recipe_ingredients_aliases_path = "CulinaryDB/04_Recipe-Ingredients_Aliases.csv"

In [30]:
recipe_details_df = pd.read_csv(recipe_details_path)
recipe_details_df.head()

,Recipe ID,Title,Source,Cuisine
0,1,5 spice vegetable fried rice,TARLA_DALAL,Indian Subcontinent
1,2,aachar aaloo,TARLA_DALAL,Indian Subcontinent
2,3,aadu lassan keri nu athanu,TARLA_DALAL,Indian Subcontinent
3,4,aaloo kofta,TARLA_DALAL,Indian Subcontinent
4,5,aaloo tamatar subzi,TARLA_DALAL,Indian Subcontinent


In [31]:
ingredients_df = pd.read_csv(ingredients_path)
ingredients_df.head()

,Aliased Ingredient Name,Ingredient Synonyms,Entity ID,Category
0,Egg,egg,0,Meat
1,Bread,bread; bun,2,Bakery
2,Rye Bread,bread-rye,3,Bakery
3,Wheaten Bread,bread-wheaten,4,Bakery
4,White Bread,bread-white; baguette,5,Bakery


In [32]:
compound_ingredients_df = pd.read_csv(compound_ingredients_path)
compound_ingredients_df = compound_ingredients_df.rename({'entity_id': 'Entity ID'}, axis=1)
compound_ingredients_df.head()

,Compound Ingredient Name,Compound Ingredient Synonyms,Entity ID,Contituent Ingredients,Category
0,Garam Masala,garam masala,2000,"black pepper, mace, cinnamon, clove, cardamom,...",Spice
1,Ginger Garlic Paste,ginger garlic paste,2001,"ginger, garlic",Spice
2,Coriander Cumin Seed Powder,coriander cumin seed powder,2002,"coriander, cumin",Spice
3,Chaat Masala,chaat masala,2003,"asafoetida, mango, black salt, cayenne, garlic...",Spice
4,Sambar Powder,sambar powder,2004,"pigeon pea, coriander, chickpea, cumin, black ...",Spice


In [33]:
recipe_ingredients_aliases_df = pd.read_csv(recipe_ingredients_aliases_path)
recipe_ingredients_aliases_df.head()

,Recipe ID,Original Ingredient Name,Aliased Ingredient Name,Entity ID
0,1,capsicum,capsicum,362
1,1,green bell pepper,pepper bell,362
2,1,soy sauce,soy sauce,291
3,1,sunflower oil,sunflower,426
4,2,buttermilk,buttermilk,61


### Data Preprocessing

#### Remove Misc Cuisines

In [34]:
recipe_details_df['Cuisine'].unique().tolist()
recipe_details_df = recipe_details_df[~recipe_details_df['Cuisine'].str.startswith('Misc.:')]
len(recipe_details_df['Cuisine'].unique().tolist())

22

#### Denormalize CulinaryDB tables

In [35]:
recipe_details_df

,Recipe ID,Title,Source,Cuisine
0,1,5 spice vegetable fried rice,TARLA_DALAL,Indian Subcontinent
1,2,aachar aaloo,TARLA_DALAL,Indian Subcontinent
2,3,aadu lassan keri nu athanu,TARLA_DALAL,Indian Subcontinent
3,4,aaloo kofta,TARLA_DALAL,Indian Subcontinent
4,5,aaloo tamatar subzi,TARLA_DALAL,Indian Subcontinent
...,...,...,...,...
45767,45768,Strawberries & Cream Frappé,EPICURIOUS,USA
45768,45769,Chocolate Whiskey Bundt Cake,EPICURIOUS,USA
45769,45770,"Wild Rice with Butternut Squash, Leeks, and Corn",EPICURIOUS,USA
45770,45771,Fruit Crumble,EPICURIOUS,USA


In [36]:
master_recipe_df = recipe_details_df.merge(recipe_ingredients_aliases_df, on='Recipe ID', how='left').drop(["Source", "Original Ingredient Name"], axis=1)
#master_recipe_df[master_recipe_df['Entity ID'] == 362.0]
master_recipe_df

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362.0
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362.0
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291.0
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426.0
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61.0
...,...,...,...,...,...
454161,45772,Enlightened Chicken Pot Pie,USA,salt,778.0
454162,45772,Enlightened Chicken Pot Pie,USA,squash yellow,493.0
454163,45772,Enlightened Chicken Pot Pie,USA,thyme,269.0
454164,45772,Enlightened Chicken Pot Pie,USA,worcestershire sauce,2031.0


In [37]:
master_recipe_df = master_recipe_df.merge(ingredients_df, on='Entity ID', how='left').drop(["Aliased Ingredient Name_y", "Ingredient Synonyms"], axis=1)
master_recipe_df

,Recipe ID,Title,Cuisine,Aliased Ingredient Name_x,Entity ID,Category
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362.0,Vegetable
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362.0,Vegetable
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291.0,Plant
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426.0,Flower
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61.0,Dairy
...,...,...,...,...,...,...
454161,45772,Enlightened Chicken Pot Pie,USA,salt,778.0,Additive
454162,45772,Enlightened Chicken Pot Pie,USA,squash yellow,493.0,Fruit
454163,45772,Enlightened Chicken Pot Pie,USA,thyme,269.0,Herb
454164,45772,Enlightened Chicken Pot Pie,USA,worcestershire sauce,2031.0,NaN


In [38]:
# Merge 'master_recipe_df' with 'compound_ingredients_df' on 'Entity ID' using a left join and drop unnecessary columns.
master_recipe_df = master_recipe_df.merge(compound_ingredients_df, on='Entity ID', how='left').drop(["Compound Ingredient Name", "Compound Ingredient Synonyms", "Contituent Ingredients"], axis=1)

# Combine the 'Category_x' and 'Category_y' columns into a new 'Category' column, filling missing values from 'Category_x'.
master_recipe_df["Category"] = np.where(master_recipe_df["Category_x"].isnull(), master_recipe_df["Category_y"], master_recipe_df["Category_x"])

# Drop the original 'Category_x' and 'Category_y' columns, as they are no longer needed.
master_recipe_df.drop(["Category_x", "Category_y"], axis=1, inplace=True)

# Rename the 'Aliased Ingredient Name_x' column to 'Aliased Ingredient Name'.
master_recipe_df = master_recipe_df.rename({'Aliased Ingredient Name_x': 'Aliased Ingredient Name'}, axis=1)

# Convert the 'Entity ID' column to Int64 type to handle nullable integer values.
master_recipe_df['Entity ID'] = master_recipe_df['Entity ID'].astype('Int64')

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID,Category
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362,Vegetable
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362,Vegetable
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291,Plant
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426,Flower
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61,Dairy
...,...,...,...,...,...,...
454161,45772,Enlightened Chicken Pot Pie,USA,salt,778,Additive
454162,45772,Enlightened Chicken Pot Pie,USA,squash yellow,493,Fruit
454163,45772,Enlightened Chicken Pot Pie,USA,thyme,269,Herb
454164,45772,Enlightened Chicken Pot Pie,USA,worcestershire sauce,2031,Dish


In [39]:
master_recipe_df[master_recipe_df.isna().any(axis=1)]

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID,Category
6954,1087,khus ka sherbat,Indian Subcontinent,NaN,<NA>,NaN
100682,11281,Italian Beef for Sandwiches,Italy,NaN,<NA>,NaN
149791,16626,Tennessee Tea,USA,NaN,<NA>,NaN
170994,18909,prodtest,Africa,NaN,<NA>,NaN
207804,21919,Decorating with Fondant,France,NaN,<NA>,NaN
226300,23711,Crispy Pepperoni Chips,Italy,NaN,<NA>,NaN
250296,25949,Recipes are temporarily unavailable. Please ch...,British Isles,NaN,<NA>,NaN
252963,26187,Calimocho,Spain,NaN,<NA>,NaN
261220,26914,Watermelon Ice Cubes,USA,NaN,<NA>,NaN
272695,27956,Chili Rubbed Rotisserie Prime Rib,USA,NaN,<NA>,NaN


In [40]:
master_recipe_df = master_recipe_df.dropna()
master_recipe_df

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID,Category
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362,Vegetable
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362,Vegetable
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291,Plant
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426,Flower
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61,Dairy
...,...,...,...,...,...,...
454161,45772,Enlightened Chicken Pot Pie,USA,salt,778,Additive
454162,45772,Enlightened Chicken Pot Pie,USA,squash yellow,493,Fruit
454163,45772,Enlightened Chicken Pot Pie,USA,thyme,269,Herb
454164,45772,Enlightened Chicken Pot Pie,USA,worcestershire sauce,2031,Dish


In [42]:
master_recipe_df["Aliased Ingredient Name"] = master_recipe_df["Aliased Ingredient Name"].str.strip()

/var/folders/tk/3mpzsdc17flggsprn40wq3380000gn/T/ipykernel_75335/3043333264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_recipe_df["Aliased Ingredient Name"] = master_recipe_df["Aliased Ingredient Name"].str.strip()


#### Check

In [43]:
recipe_ingredients_aliases_df.size

1825116

In [44]:
master_recipe_df.size

2724858

In [45]:
master_recipe_df.isnull().values.any()

False

## FlavorDB

### Acquiring Data
Scraping the data from FlavorDB
> https://vchoo.github.io/

The general steps to data scraping are:

1. Download the JSON files.
2. Process the JSON data.
3. Clean the DataFrame.

In [46]:
# for basic data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import json

### 1. Download JSON files

```
import os
import requests
import json

# Get the current working directory
cwd_dir = os.getcwd()

# Create a directory called "FlavorDB" within the current working directory
db_dir = os.path.join(cwd_dir, "FlavorDB")

# Check if the "FlavorDB" directory exists, and if not, create it
if not os.path.exists(db_dir):
    os.makedirs(db_dir)

if __name__ == "__main__":
    # Loop through a range of 1000 entities (indexed from 0 to 999)
    for index in range(1000):
        try:
            # Compose the URL to fetch data for the current entity index
            url = "https://cosylab.iiitd.edu.in/flavordb2/entities_json?id=%d" % index

            # Send a GET request to the URL and parse the response as JSON
            data = requests.get(url).json()

            # Convert the JSON data to a string for writing to a file
            obj = json.dumps(data)

            # Extract the entity name from the JSON data
            name_key = "entity_alias_readable"
            name = data[name_key]

            # Print the progress of downloading data for the current entity
            print("Downloading %s data" % name)

            # Create a file name using the current index and save the JSON data to that file
            file_name = os.path.join(db_dir, "%d.json" % index)

            with open(file_name, 'w') as f:
                json.dump(data, f, indent=4, sort_keys=True)

            index += 1  # Move to the next entity index
        except:
            # If there is an exception (e.g., the entity doesn't exist), continue to the next index
            continue

    print("Done")

```

### 2. Process JSON data

In [47]:
# the names of the "columns" in the raw JSON objects
def flavordb_entity_cols():
    return ['entity_id', 'entity_alias_readable', 'entity_alias_synonyms','natural_source_name', 'category_readable', 'molecules']


# what we want to rename the JSON object "columns" to
def flavordb_df_cols():
    return ['entity id', 'alias', 'synonyms','scientific name', 'category', 'molecules']


# "subcolumns" in the "molecules" column that we are interested in
def molecules_df_cols():
    return ['pubchem id', 'common name', 'flavor profile', 'cas id']   

In [48]:
import os
import json

# Translates the JSON file at the specified dir into a dictionary.
def get_flavordb_entity(x):
    # Construct the file path for the JSON file based on the input 'x'.
    file_path = os.path.join('FlavorDB', str(x) + '.json')
    
    # Check if the file exists.
    if os.path.isfile(file_path):
        # If the file exists, open it and load its contents as a dictionary.
        with open(file_path, "r") as json_file:
            return json.load(json_file)
    else:
        # If the file does not exist, return None to indicate that no data was found.
        return None


In [49]:
def clean_flavordb_dataframes(flavor_df, molecules_df):
    """
    Helps ensure consistent intra-column typing and converts all strings to lowercase.

    Args:
        flavor_df (DataFrame): DataFrame containing flavor data.
        molecules_df (DataFrame): DataFrame containing molecules data.

    Returns:
        list: A list of two DataFrames, each grouped by specific columns.
    """

    strtype = type('')  # Define the type for strings
    settype = type(set())  # Define the type for sets

    # Convert 'alias', 'scientific name', and 'category' columns to lowercase strings
    for column in ['alias', 'scientific name', 'category']:
        flavor_df[column] = [elem.strip().lower() if isinstance(elem, strtype) else '' for elem in flavor_df[column]]

    # Convert 'synonyms' column to a set of lowercase strings
    def map_to_synonyms_set(elem):
        """
        Converts an element to a set of lowercase strings.

        Args:
            elem: Element to be converted.

        Returns:
            set: A set of lowercase strings.
        """
        if isinstance(elem, settype):
            return elem
        elif isinstance(elem, strtype):
            if elem == '':
                return set()
            # If it's a string of a set, convert it to a set
            if elem[0] == '{' and elem[-1] == '}':
                return eval(elem)
            else:
                # Else, it's probably directly from the source
                return set(elem.strip().lower().split(', '))
        else:
            return set()

    flavor_df['synonyms'] = [map_to_synonyms_set(elem) for elem in flavor_df['synonyms']]

    # Convert 'flavor profile' column in molecules_df to a set of lowercase strings
    molecules_df['flavor profile'] = [set([x.strip().lower() for x in elem]) for elem in molecules_df['flavor profile']]

    # Group by 'entity id' and 'pubchem id', respectively, and keep the first row of each group
    cleaned_flavor_df = flavor_df.groupby('entity id').first().reset_index()
    cleaned_molecules_df = molecules_df.groupby('pubchem id').first().reset_index()

    return [cleaned_flavor_df, cleaned_molecules_df]


In [50]:
import pandas as pd

def get_flavordb_dataframes():
    """
    Read JSON data, convert it to DataFrames, and clean them.
    
    Returns DataFrames for both foods and molecules, as well as missing JSON entries.
    """
    
    # Initialize an empty list to store data from Flavordb JSON
    flavordb_data = []
    
    # Initialize an empty dictionary to store molecules' data
    molecules_dict = {}
    
    # Get the columns for the Flavordb entities
    flavordb_cols = flavordb_entity_cols()

    # Loop through Flavordb entities from 0 to 999 (a total of 1000 entries)
    for i in range(0, 1000):
        try:
            # Get the Flavordb entity for the current index
            fdbe = get_flavordb_entity(i)
            
            # If the entity is None (missing entry), continue to the next iteration
            if fdbe is None:
                continue
            
            # Extract specific columns from the Flavordb entity and store them in a list
            flavordb_series = [fdbe[k] for k in flavordb_cols[:-1]]
            flavordb_series.append(set([m['pubchem_id'] for m in fdbe['molecules']]))
            flavordb_data.append(flavordb_series)

            # Process and store molecule data in the dictionary
            for m in fdbe['molecules']:
                if m['pubchem_id'] not in molecules_dict:
                    molecules_dict[m['pubchem_id']] = [m['common_name'], set(m['flavor_profile'].split('@')), set(m['cas_id'].split('@'))]
        except Exception:
            # Handle any exceptions during JSON retrieval (broad exception handling, better to specify the exception)
            continue

    # Create a DataFrame from the Flavordb data
    flavordb_df = pd.DataFrame(flavordb_data, columns=flavordb_df_cols())
    
    # Create a DataFrame from the molecules dictionary
    molecules_df = pd.DataFrame([[k, v[0], v[1], v[2]] for k, v in molecules_dict.items()], columns=molecules_df_cols())

    # Clean the Flavordb and molecules DataFrames
    flavordb_df, molecules_df = clean_flavordb_dataframes(flavordb_df, molecules_df)

    # Return the cleaned DataFrames
    return [flavordb_df, molecules_df]


In [51]:
flavor_df, molecules_df = get_flavordb_dataframes()

In [53]:
flavor_df.head()

,entity id,alias,synonyms,scientific name,category,molecules
0,0,egg,{egg},chicken,animal product,"{6274, 5311110, 644104, 9609, 18827, 527, 1960..."
1,1,bakery products,{bakery products},poacceae,bakery,"{27457, 7976, 31252, 26808, 22201, 26331}"
2,2,bread,{bread},poacceae,bakery,"{1031, 1032, 644104, 527, 8723, 31260, 15394, ..."
3,3,rye bread,{rye bread},rye,bakery,"{644104, 7824, 643731, 8468, 1049, 5372954, 80..."
4,4,wheaten bread,"{soda farls, soda scones}",wheat,bakery,"{5365891, 6915, 1146, 12170, 8082, 31251, 7958..."


In [54]:
flavor_df.shape

(936, 6)

In [55]:
molecules_df.head()

,pubchem id,common name,flavor profile,cas id
0,4,1-Aminopropan-2-ol,{fishy},"{2799-17-9, 78-96-6, 2799-16-8}"
1,49,3-Methyl-2-oxobutanoic acid,{fruity},{759-05-7}
2,58,2-oxobutanoic acid,"{creamy, lactonic, brown, sweet, caramel}",{600-18-0}
3,70,4-Methyl-2-oxovaleric acid,{fruity},{816-66-0}
4,72,"3,4-Dihydroxybenzoic Acid","{phenolic, mild, balsamic}",{99-50-3}


In [56]:
molecules_df.shape

(1788, 4)

### Alignment between master_recipe_df & flavors_df

ID which ingredients are missing from FlavorDB

In [57]:
# Printing the number of unique ingredients in the master_recipe_df
print("Number of ingredients in master_recipe_df: %s" % len(master_recipe_df['Entity ID'].unique()))

# Printing the number of unique ingredients in the flavors_df
print("Number of ingredients in flavors_df: %s" % len(flavor_df['entity id'].unique()))

# Finding missing ingredient IDs (present in master_recipe_df but not in flavors_df)
missing_ingredients_ids = list(set(master_recipe_df['Entity ID'].unique()) - set(flavor_df['entity id'].unique()))

# Printing the number of missing ingredient IDs
print("Missing ingredient IDs: %s" % len(missing_ingredients_ids))

# Grouping and counting missing ingredient names in master_recipe_df, sorting in descending order, and displaying top 15 missing ingredients
result_df = master_recipe_df[master_recipe_df['Entity ID'].isin(missing_ingredients_ids)].groupby(["Aliased Ingredient Name", "Entity ID"])["Aliased Ingredient Name"].count().reset_index(name='count').sort_values(['count'], ascending=False)[:15]
print(result_df)


Number of ingredients in master_recipe_df: 695
Number of ingredients in flavors_df: 936
Missing ingredient IDs: 113


,Aliased Ingredient Name,Entity ID,count
50,flour,2056,9294
137,vegetable oil,2089,5584
81,lemon juice,984,4036
22,cayenne,992,3709
116,salt pepper,2022,3129
9,baking powder,996,2237
86,mayonnaise,2021,1546
10,baking soda,996,1387
132,tomato paste,2064,1158
142,worcestershire sauce,2031,1148


In [58]:
missing_ingredients_df = master_recipe_df[master_recipe_df['Entity ID'].isin(missing_ingredients_ids)][["Entity ID","Aliased Ingredient Name"]].drop_duplicates().reset_index(drop=True)

Read the FlavorNetwork dataset

In [59]:
# Read ingredient information from 'ingr_info.tsv' file into a DataFrame.
# Use tab ('\t') as the separator and consider the first row as the header.
ingr_info_df = pd.read_csv('FlavorNetwork/ingr_info.tsv', sep='\t', header=0)

# Clean up the "ingredient name" column by replacing underscores ('_') with spaces (' ').
ingr_info_df["ingredient name"] = ingr_info_df["ingredient name"].str.replace(r'_', ' ')

# Read ingredient composition data from 'ingr_comp.tsv' file into another DataFrame.
# Use tab ('\t') as the separator and consider the first row as the header.
ingr_comp_df = pd.read_csv('FlavorNetwork/ingr_comp.tsv', sep='\t', header=0)

# Read component information from 'comp_info.tsv' file into a third DataFrame.
# Use tab ('\t') as the separator and consider the first row as the header.
comp_info_df = pd.read_csv('FlavorNetwork/comp_info.tsv', sep='\t', header=0)


Break apart the molecules dataset to get the cas ids

In [1390]:
molecules_tmp_df = molecules_df.copy()
molecules_tmp_df['cas id'] = molecules_tmp_df['cas id'].apply(list)
molecules_tmp_df = molecules_tmp_df.explode('cas id')
molecules_tmp_df

,pubchem id,common name,flavor profile,cas id
0,4,1-Aminopropan-2-ol,{fishy},78-96-6
0,4,1-Aminopropan-2-ol,{fishy},2799-16-8
0,4,1-Aminopropan-2-ol,{fishy},2799-17-9
1,49,3-Methyl-2-oxobutanoic acid,{fruity},759-05-7
2,58,2-oxobutanoic acid,"{caramel, brown, sweet, lactonic, creamy}",600-18-0
...,...,...,...,...
1783,76319594,(-)-alpha-Copaene,{},3856-25-5
1784,91865027,UNII-S32G48306Y,{},26184-88-3
1785,92043553,Jasmine Oil,{},8022-96-6
1786,92043681,"Glycerides, palm-oil mono- and di-, hydrogenat...",{},91052-70-9


Denormalize the Flavor Network dataset

In [1391]:
# Merge the 'ingr_info_df' DataFrame with 'ingr_comp_df' DataFrame using the 'id' and '# ingredient id' columns as the keys,
# keeping only the left rows. Then, drop the '# ingredient id' column from the resulting DataFrame.
ingr_info_df = ingr_info_df.merge(ingr_comp_df, left_on='# id', right_on="# ingredient id", how='left').drop(columns=['# ingredient id'])

# Convert the 'compound id' column to nullable integers (Int64).
ingr_info_df['compound id'] = ingr_info_df['compound id'].astype('Int64')

# Convert the '# id' column in 'comp_info_df' to nullable integers (Int64).
comp_info_df['# id'] = comp_info_df['# id'].astype('Int64')

# Merge the 'ingr_info_df' DataFrame with 'comp_info_df' DataFrame using the 'compound id' and '# id' columns as the keys,
# keeping only the left rows. Then, drop the '# id_y' column from the resulting DataFrame and rename '# id_x' to '# id'.
ingr_info_df = ingr_info_df.merge(comp_info_df, left_on="compound id", right_on='# id', how='left').drop(columns=['# id_y']).rename(columns={'# id_x': '# id'})


,# id,ingredient name,category,compound id,Compound name,CAS number
0,0,magnolia tripetala,flower,995,diacetyl,431-03-8
1,1,calyptranthes parriculata,plant,921,citral_(neral),5392-40-5
2,2,chamaecyparis pisifera oil,plant derivative,1018,undecanoic_acid,112-37-8
3,3,mackerel,fish/seafood,272,"2,4-nonadienal",6750-03-4
4,3,mackerel,fish/seafood,275,lauric_acid,143-07-7
...,...,...,...,...,...,...
36781,1529,munster cheese,dairy,1102,2-heptanol,543-49-7
36782,1529,munster cheese,dairy,268,isoamyl_hexanoate,2198-61-0
36783,1529,munster cheese,dairy,821,isoamyl_butyrate,106-27-4
36784,1529,munster cheese,dairy,554,n-nonanal,124-19-6


Merge molecules_df onto it to get the pubchem id

In [1392]:
ingr_info_df = ingr_info_df.merge(molecules_tmp_df[["pubchem id","cas id"]], left_on="CAS number", right_on="cas id", how='left').drop(columns=['cas id'])
ingr_info_df['pubchem id'] = ingr_info_df['pubchem id'].astype('Int64')
ingr_info_df

,# id,ingredient name,category,compound id,Compound name,CAS number,pubchem id
0,0,magnolia tripetala,flower,995,diacetyl,431-03-8,650
1,1,calyptranthes parriculata,plant,921,citral_(neral),5392-40-5,638011
2,1,calyptranthes parriculata,plant,921,citral_(neral),5392-40-5,643779
3,2,chamaecyparis pisifera oil,plant derivative,1018,undecanoic_acid,112-37-8,8180
4,3,mackerel,fish/seafood,272,"2,4-nonadienal",6750-03-4,5283339
...,...,...,...,...,...,...,...
38738,1529,munster cheese,dairy,1102,2-heptanol,543-49-7,10976
38739,1529,munster cheese,dairy,268,isoamyl_hexanoate,2198-61-0,16617
38740,1529,munster cheese,dairy,821,isoamyl_butyrate,106-27-4,7795
38741,1529,munster cheese,dairy,554,n-nonanal,124-19-6,31289


Enrich the missing_ingredients_df with the ids of the molecules

In [1393]:
# Function to get the category of an ingredient from the ingr_info_df DataFrame
def apply_category(ingredient_name):
    try:
        # Filter ingr_info_df by ingredient name and get the category
        cat_name = ingr_info_df[ingr_info_df["ingredient name"] == ingredient_name]["category"].iloc[0]
        # Map some specific categories to simplified categories
        if cat_name == "fish/seafood":
            return "seafood"
        elif cat_name == "beverage alcoholic":
            return "beverage alcoholic"
        else:
            return cat_name
    except:
        # Return None if there's an error or if the ingredient is not found
        return None

# Function to get the set of pubchem ids associated with an ingredient from the ingr_info_df DataFrame
def apply_molecules(ingredient_name):
    # Filter ingr_info_df by ingredient name and drop rows with missing pubchem ids
    filtered_df = ingr_info_df[ingr_info_df['ingredient name'] == ingredient_name].dropna(subset=['pubchem id'])
    # Get the set of unique pubchem ids
    pubchem_ids = set(filtered_df['pubchem id'])
    return pubchem_ids

# Get a list of missing ingredient names that exist in both missing_ingredients_df and ingr_info_df
missing_ingredients_names = list(set(missing_ingredients_df["Aliased Ingredient Name"].unique()).intersection(set(ingr_info_df["ingredient name"].unique())))

# Filter missing_ingredients_df to only include rows with missing ingredient names
missing_ingredients_df = missing_ingredients_df[missing_ingredients_df["Aliased Ingredient Name"].isin(missing_ingredients_names)]

# Apply the apply_category function to get the category for each missing ingredient
missing_ingredients_df["category"] = missing_ingredients_df["Aliased Ingredient Name"].apply(apply_category)

# Apply lambda function to get the set of synonyms for each missing ingredient
missing_ingredients_df["synonyms"] = missing_ingredients_df["Aliased Ingredient Name"].apply(lambda x: set(ingr_info_df[ingr_info_df["ingredient name"] == x]["ingredient name"]))

# Apply the apply_molecules function to get the set of molecules (pubchem ids) for each missing ingredient
missing_ingredients_df["molecules"] = missing_ingredients_df["Aliased Ingredient Name"].apply(apply_molecules)

# Fill the "scientific name" column with the same values as the "Aliased Ingredient Name" column
missing_ingredients_df["scientific name"] = missing_ingredients_df["Aliased Ingredient Name"]

# Rename the columns to have more readable names
missing_ingredients_df.rename(columns={"Entity ID": "entity id", "Aliased Ingredient Name": "alias", "synonyms": "synonyms", "category": "category", "molecules": "molecules", "scientific name": "scientific name"}, inplace=True)


Add the complete missing_ingredients_df to flavor_df

In [1394]:
flavor_df = flavor_df.append(missing_ingredients_df[["entity id","alias","synonyms","category","molecules","scientific name"]]).reset_index(drop=True)

Check

In [1395]:
flavor_df[flavor_df["entity id"].isin(missing_ingredients_df["entity id"].unique())]

,entity id,alias,synonyms,scientific name,category,molecules
936,992,cayenne,{cayenne},cayenne,spice,"{5362819, 7559, 102667, 1548943, 7824, 7826, 5..."
937,982,coconut oil,{coconut oil},coconut oil,plant derivative,"{13187, 8163, 11622, 12777, 12844, 335, 8175, ..."
938,993,yeast,{yeast},yeast,plant,{6202}
939,2088,crab,{crab},crab,seafood,"{8193, 1146, 18827, 19602, 1049, 8093, 798, 11..."
940,984,lemon juice,{lemon juice},lemon juice,fruit,"{9017, 7909, 62367}"
941,2089,vegetable oil,{vegetable oil},vegetable oil,plant derivative,{7762}
942,2071,mackerel,{mackerel},mackerel,seafood,"{5283329, 90246, 1031, 5283335, 11527, 1032, 5..."
943,980,apple juice,{apple juice},apple juice,fruit,"{6276, 12293, 10886, 135, 1032, 11529, 12294, ..."
944,1000,gelatin,{gelatin},gelatin,animal product,{}
945,2096,catfish,{catfish},catfish,seafood,"{5283329, 5311110, 1031, 90246, 5283335, 11527..."


In [1396]:
len(flavor_df[flavor_df["entity id"] == 0]["molecules"].to_list()[0])

55

In [1397]:
test = list(flavor_df[flavor_df["entity id"] == 0]["molecules"].to_list()[0])
len(molecules_df[molecules_df["pubchem id"].isin(test)])

55

In [1398]:
str(set(flavor_df['category']))

"{'vegetable', 'fungus', 'herb', 'beverage', 'fish', 'animal product', 'cereal', 'berry', 'dairy', 'fruit', 'legume', 'bakery', 'fruit-berry', 'seafood', 'fruit citrus', 'maize', 'dish', 'cabbage', 'alcoholic beverage', 'vegetable stem', 'flower', 'seed', 'gourd', 'additive', 'vegetable tuber', 'essential oil', 'plant', 'vegetable root', 'nut', 'vegetable fruit', 'fruit essence', 'beverage caffeinated', 'beverage alcoholic', 'meat', 'plant derivative', 'spice'}"

### Feature Engineering

#### Group

"We still have some more data cleaning to do. What's the difference between a vegetable, fruit, and vegetable fruit? We're mostly interested in the ingredients list, not in finished products like cornbread, so we'll keep only raw ingredients, remove a few outliers, and give them each a food group." (Choo, V.)

In [1399]:
def food_groups():
    """
    Return a set of food groups.
    """
    return set([
        'grain', 'vegetable', 'fruit', 'protein',
        'dairy', 'fat', 'sugar', 'seasoning',
        'beverage', 'alcohol'
    ])

def get_food_group(food, category):
    """
    Maps each food category to a food group.

    The food groups include the main five: grain, vegetable, fruit, protein,
    dairy, fat, and sugar. However, they also include others: seasonings,
    beverages, alcohol.

    Args:
        food (str): The specific food item.
        category (str): The category of the food item.

    Returns:
        str or None: The food group to which the food item belongs, or None if it doesn't belong to any group.
    """
    out = None

    # Categorize the food item based on its category
    if category in ['bakery', 'vegetable tuber', 'cereal']:
        out = 'grain'
    elif category in [
        'flower', 'fungus', 'plant', 'cabbage',
        'vegetable fruit', 'herb', 'gourd', 'vegetable'
    ]:
        out = 'vegetable'
    elif category in [
        'fruit-berry', 'berry', 'fruit', 'fruit citrus'
    ]:
        out = 'fruit'
    elif category in [
        'legume', 'nut', 'seed', 'seafood', 'fish', 'meat'
    ]:
        out = 'protein'
    elif category in ['dairy']:
        out = 'dairy'
    elif category in [
        'fruit essence', 'additive', 'spice', 'essential oil'
    ]:
        out = 'seasoning'
    elif category in ['beverage alcoholic']:
        out = 'alcohol'
    elif 'beverage' in category:
        out = 'beverage'
    # Handle special cases within the 'maize' category
    elif category == 'maize':
        if food in ['corn', 'sweetcorn']:
            out = 'vegetable'
        elif food in ['cornbread', 'corn grits', 'popcorn']:
            out = 'grain'
        elif food == 'corn oil':
            out = 'fat'
    # Handle special cases within the 'plant derivative' category
    elif category == 'plant derivative':
        if (any(x in food for x in ['sauce', 'vinegar', 'cocoa'])
                or food in ['creosote', 'storax']):
            out = 'seasoning'
        elif 'seed' in food or food == 'peanut butter':
            out = 'protein'
        elif any([x in food for x in ['butter', 'oil']]):
            out = 'fat'
        elif food == 'fermented tea':
            out = 'beverage'
        elif food in ['honey', 'chocolate', 'chocolate spread']:
            out = 'sugar'
        elif food == 'macaroni':
            out = 'grain'
        elif food in ['jute', 'tofu']:
            out = 'vegetable'
        elif food == 'soy yogurt':
            out = 'dairy'
    # Handle special cases within the 'additive' category
    elif category == 'additive':
        if 'sugar' in food or food in [
            'fruit preserve', 'syrup', 'icing', 'molasses'
        ]:
            out = 'sugar'
        elif 'margarine' in food or food in ['cooking oil', 'shortening']:
            out = 'fat'
        elif food in ['sauce', 'gelatin dessert', 'spread', 'topping', 'water']:
            out = None
        elif food == 'stuffing':
            out = 'grain'
        else:
            out = 'seasoning'

    # Check for specific food items that belong to the 'fat' group
    if (
        any([
            food == x + ' oil'
            for x in ['soybean', 'cooking', 'fish', 'peanut', 'canola', 'corn']
        ])
        or food in ['butter', 'ghee']
        or (' butter' in food and food != 'peanut butter')
        or 'margarine' in food
    ):
        out = 'fat'
    # Check for specific food items that belong to the 'sugar' group
    elif food in [
        'sugar', 'honey', 'molasses', 'agave', 'dulce de leche'
    ]:
        out = 'sugar'
    # Check for specific food items that belong to the 'vegetable' group
    elif food in ['irish moss', 'kelp', 'kombu', 'wakame']:
        out = 'vegetable'
    elif food in ['butternut squash', 'winter squash', 'japanese pumpkin']:
        out = 'vegetable'
    # Check for specific food items that belong to the 'sugar' group (sweet desserts)
    elif food in ['sweet custard', 'candy bar', 'chocolate mousse', 'fudge']:
        out = 'sugar'
    # Check for specific food items that belong to the 'seasoning' group (cocoa-related items)
    elif 'cocoa' in food:
        out = 'seasoning'

    return out


# The DataFrame is assumed to have columns 'alias' (the specific food item) and 'category' (the category of the item).
ridf = flavor_df.copy()
ridf['Group'] = [
    get_food_group(ridf.at[i, 'alias'], ridf.at[i, 'category'])
    for i in ridf.index
]
# Drop any rows where the group is None (i.e., the food item does not belong to any group).
# If you want to keep the None values, you can comment this line.
# ridf = ridf[[g is not None for g in ridf['group']]]
ridf = ridf.reset_index()
ridf.head()


,index,entity id,alias,synonyms,scientific name,category,molecules,Group
0,0,0,egg,{egg},chicken,animal product,"{6274, 5311110, 644104, 9609, 18827, 527, 1960...",None
1,1,1,bakery products,{bakery products},poacceae,bakery,"{27457, 7976, 31252, 26808, 22201, 26331}",grain
2,2,2,bread,{bread},poacceae,bakery,"{1031, 1032, 644104, 527, 8723, 31260, 15394, ...",grain
3,3,3,rye bread,{rye bread},rye,bakery,"{644104, 7824, 643731, 8468, 1049, 5372954, 80...",grain
4,4,4,wheaten bread,"{soda farls, soda scones}",wheat,bakery,"{5365891, 6915, 1146, 12170, 8082, 31251, 7958...",grain


In [1400]:
str(set(ridf['category']))

"{'vegetable', 'fungus', 'herb', 'beverage', 'fish', 'animal product', 'cereal', 'berry', 'dairy', 'fruit', 'legume', 'bakery', 'fruit-berry', 'seafood', 'fruit citrus', 'maize', 'dish', 'cabbage', 'alcoholic beverage', 'vegetable stem', 'flower', 'seed', 'gourd', 'additive', 'vegetable tuber', 'essential oil', 'plant', 'vegetable root', 'nut', 'vegetable fruit', 'fruit essence', 'beverage caffeinated', 'beverage alcoholic', 'meat', 'plant derivative', 'spice'}"

In [1401]:
ridf.rename(columns={"entity id":"Entity ID","alias":"Aliased Ingredient Name","synonyms":"Synonyms","category":"Category","molecules":"Molecules","scientific name":"Scientific Name"}, inplace=True)

#### Dietary Style

Taken from RecipeDB. (Kim, J. Y., Glober, N., & Varshney, L. R. (2019))

Rules were created for designating a dietary style based on the ingredient category and/or ingredient(s) used in a recipe. For each dietary style, the rules were formed to specify categories of ingredients that were restricted and the ones mandatory. For a recipe to be associated with a dietary style, it needed to have no ingredient belonging to the dietary style’s list of restricted categories while having at least one or more ingredients belonging to each of the dietary style’s list of mandatory categories. As a precaution, any recipe containing an ingredient under the category ‘bakery’ was assumed to have eggs in it. Also, a recipe was not mapped to any dietary style if it had an ingredient(s) of the ‘dish’ category. 

Following were the rules used: Vegan (Exclude: meat, eggs, dairy, fish, seafood, dish); Pescetarian (Include: fish or seafood and Exclude: meat, dairy, dish); Lacto-Vegetarian (Include: dairy and Exclude: meat, eggs, fish, seafood, dish); Ovo-Vegetarian (Include: egg and Exclude meat, fish, seafood, dairy, dish); and Ovo-Lacto Vegetarian (Include: egg and dairy and Exclude: meat, fish, seafood, dish).

In [1402]:
ingredients_df['Category'].unique().tolist()

['Meat',
 'Bakery',
 'Beverage',
 'Beverage Alcoholic',
 'Cereal',
 'Maize',
 'Dairy',
 'Essential Oil',
 'Fruit',
 'Seafood',
 'Fish',
 'Flower',
 'Fungus',
 'Herb',
 'Dish',
 'Nuts & Seed',
 'Legume',
 'Plant',
 'Spice',
 'Vegetable',
 'Additive']

In [1404]:
# Splitting the ingredients into categories
tmp_master_recipe_df = master_recipe_df[["Entity ID", "Category"]].drop_duplicates().reset_index(drop=True)

tmp_dietary_df = master_recipe_df[['Recipe ID']].drop_duplicates().reset_index(drop=True)

In [1405]:
# Define a function to apply dietary rules based on the ingredients of a recipe
def apply_dietary_rules(recipe_id):
    
    # Create a set to store unique categories of ingredients in the recipe
    recipe_category_set = set()
    
    # Iterate through each ingredient ID in the recipe
    for ingredient_id in recipe_ingredients_aliases_df[recipe_ingredients_aliases_df['Recipe ID'] == recipe_id]["Entity ID"]:
        # Add the category of the ingredient to the set
        recipe_category_set.add(tmp_master_recipe_df[tmp_master_recipe_df["Entity ID"] == ingredient_id].iloc[0]["Category"])
    
    # Check dietary rules based on the categories present in the recipe ingredients
    if "Meat" not in recipe_category_set and \
            "Fish" not in recipe_category_set and \
                "Seafood" not in recipe_category_set and \
                    "Eggs" not in recipe_category_set and \
                        "Dairy" not in recipe_category_set and \
                            "Dish" not in recipe_category_set:
        return "Vegan"
    elif "Meat" not in recipe_category_set and \
            ("Fish" in recipe_category_set or \
                "Seafood" in recipe_category_set) and \
                        "Dairy" not in recipe_category_set and \
                            "Dish" not in recipe_category_set:
        return "Pescetarian"
    elif "Meat" not in recipe_category_set and \
            "Fish" not in recipe_category_set and \
                "Seafood" not in recipe_category_set and \
                    "Eggs" not in recipe_category_set and \
                        "Dairy" in recipe_category_set and \
                            "Dish" not in recipe_category_set:
        return "Lacto-Vegetarian"
    elif "Meat" not in recipe_category_set and \
            "Fish" not in recipe_category_set and \
                "Seafood" not in recipe_category_set and \
                    "Eggs" in recipe_category_set and \
                        "Dairy" not in recipe_category_set and \
                            "Dish" not in recipe_category_set:
        return "Ovo-Vegetarian"
    elif "Meat" not in recipe_category_set and \
            "Fish" not in recipe_category_set and \
                "Seafood" not in recipe_category_set and \
                    "Eggs" in recipe_category_set and \
                        "Dairy" in recipe_category_set and \
                            "Dish" not in recipe_category_set:
        return "Ovo-Lacto Vegetarian"
    elif "Dish" not in recipe_category_set:
        return "Non-Vegetarian"
    # If no specific rules match, it falls under "Other" category (Optional)
    # else:
    #     return "Other"

# Apply the dietary rules to each recipe ID in the DataFrame and create a new column "Dietary Style"
tmp_dietary_df['Dietary Style'] = tmp_dietary_df['Recipe ID'].swifter.apply(apply_dietary_rules)


Dask Apply:   0%|          | 0/20 [00:00<?, ?it/s]

In [1406]:
master_recipe_df = master_recipe_df.merge(tmp_dietary_df, on='Recipe ID', how='left')
del tmp_dietary_df
master_recipe_df

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID,Category,Dietary Style
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362,Vegetable,Vegan
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362,Vegetable,Vegan
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291,Plant,Vegan
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426,Flower,Vegan
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61,Dairy,Lacto-Vegetarian
...,...,...,...,...,...,...,...
454138,45772,Enlightened Chicken Pot Pie,USA,salt,778,Additive,None
454139,45772,Enlightened Chicken Pot Pie,USA,squash yellow,493,Fruit,None
454140,45772,Enlightened Chicken Pot Pie,USA,thyme,269,Herb,None
454141,45772,Enlightened Chicken Pot Pie,USA,worcestershire sauce,2031,Dish,None


## Save datasets for analysis

In [1411]:
ridf.head()

,index,Entity ID,Aliased Ingredient Name,Synonyms,Scientific Name,Category,Molecules,Group
0,0,0,egg,{egg},chicken,animal product,"{6274, 5311110, 644104, 9609, 18827, 527, 1960...",None
1,1,1,bakery products,{bakery products},poacceae,bakery,"{27457, 7976, 31252, 26808, 22201, 26331}",grain
2,2,2,bread,{bread},poacceae,bakery,"{1031, 1032, 644104, 527, 8723, 31260, 15394, ...",grain
3,3,3,rye bread,{rye bread},rye,bakery,"{644104, 7824, 643731, 8468, 1049, 5372954, 80...",grain
4,4,4,wheaten bread,"{soda farls, soda scones}",wheat,bakery,"{5365891, 6915, 1146, 12170, 8082, 31251, 7958...",grain


In [1407]:
ridf.to_pickle('ProcessedDBs/flavordb_ridf.pkl')

In [1412]:
master_recipe_df.head()

,Recipe ID,Title,Cuisine,Aliased Ingredient Name,Entity ID,Category,Dietary Style
0,1,5 spice vegetable fried rice,Indian Subcontinent,capsicum,362,Vegetable,Vegan
1,1,5 spice vegetable fried rice,Indian Subcontinent,pepper bell,362,Vegetable,Vegan
2,1,5 spice vegetable fried rice,Indian Subcontinent,soy sauce,291,Plant,Vegan
3,1,5 spice vegetable fried rice,Indian Subcontinent,sunflower,426,Flower,Vegan
4,2,aachar aaloo,Indian Subcontinent,buttermilk,61,Dairy,Lacto-Vegetarian


In [1408]:
master_recipe_df.to_pickle('ProcessedDBs/master_recipe_df.pkl')